In [39]:
import torch, platform
print("PyTorch:", torch.__version__, "| CUDA:", torch.cuda.is_available(), "| Python:", platform.python_version())


PyTorch: 2.8.0+cu126 | CUDA: True | Python: 3.12.11


In [40]:
!pip install -q transformers datasets evaluate scikit-learn accelerate
!pip -q install -U "transformers>=4.56.1" "datasets>=2.14" accelerate

In [41]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
# Set the path to the file you'd like to load
file_path = "1429_1.csv"
# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "datafiniti/consumer-reviews-of-amazon-products",
  file_path,
)
df.head(3)

/tmp/ipython-input-340698197.py:6: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'consumer-reviews-of-amazon-products' dataset.


/usr/local/lib/python3.12/dist-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ


In [42]:
df.keys()

Index(['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer',
       'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username'],
      dtype='object')

In [43]:
#checking for NaN in the dataset
df.isnull().sum()

,0
id,0
name,6760
asins,2
brand,0
categories,0
keys,0
manufacturer,0
reviews.date,39
reviews.dateAdded,10621
reviews.dateSeen,0


In [44]:
#dropping unnecessary columns
columns_to_drop = [
    'id', 'dateAdded','reviews.didPurchase', 'brand','dateUpdated', 'asins', 'primaryCategories',
    'imageURLs', 'keys', 'manufacturer', 'manufacturerNumber',
    'reviews.date', 'reviews.dateSeen','reviews.userCity',
       'reviews.userProvince'
    'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful',
    'reviews.sourceURLs',
       'reviews.userProvince','reviews.username','name', 'categories', 'sourceURLs','reviews.dateAdded'
]

# Get the columns that actually exist in the DataFrame
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

# Drop only the existing columns
df = df.drop(columns=existing_columns_to_drop)

In [45]:
df.keys()

Index(['reviews.doRecommend', 'reviews.rating', 'reviews.text',
       'reviews.title'],
      dtype='object')

In [46]:
df.head()

,reviews.doRecommend,reviews.rating,reviews.text,reviews.title
0,True,5.0,This product so far has not disappointed. My c...,Kindle
1,True,5.0,great for beginner or experienced person. Boug...,very fast
2,True,5.0,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.
3,True,4.0,I've had my Fire HD 8 two weeks now and I love...,Good!!!
4,True,5.0,I bought this for my grand daughter when she c...,Fantastic Tablet for kids


In [47]:
df.keys()

Index(['reviews.doRecommend', 'reviews.rating', 'reviews.text',
       'reviews.title'],
      dtype='object')

**IMPORTING NECECARY LIBUARIES**




In [48]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import torch

In [49]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)
from datasets import Dataset

In [50]:

# Keeping only the columns we need
# Common columns in Datafiniti Amazon reviews:
#  reviews.text and reviews.rating
needed_cols = []
if 'reviews.text' in df.columns:
    needed_cols.append('reviews.text')
if 'reviews.rating' in df.columns:
    needed_cols.append('reviews.rating')

assert len(needed_cols) == 2, f"Expected 'reviews.text' and 'reviews.rating' in df.columns, got: {df.columns.tolist()}"

df = df[needed_cols].copy()

In [51]:
# Basic cleaning

def clean_text(s: str) -> str:
    s = re.sub(r'<br\s*/?>', ' ', str(s))      # remove HTML breaks
    s = re.sub(r'\s+', ' ', s).strip()         # normalize spaces
    return s

df['reviews.text'] = df['reviews.text'].fillna('').map(clean_text)
df['reviews.rating'] = pd.to_numeric(df['reviews.rating'], errors='coerce').astype('Int64')

#Drop empties & invalid ratings
df = df.dropna(subset=['reviews.rating'])
df = df[df['reviews.text'].str.len() > 0]
df = df[df['reviews.rating'].between(1, 5)]


In [52]:
# 3) Map stars → 3 sentiment classes
#    1-2 = 0 (negative), 3 = 1 (neutral), 4-5 = 2 (positive)
# ------------------------------
def map_star_to_label(star):
    if star in (1, 2):
        return 0
    if star == 3:
        return 1
    return 2  # 4 or 5

In [53]:
df['label'] = df['reviews.rating'].map(map_star_to_label).astype(int)
df['text']  = df['reviews.text']  # rename for HF Datasets convention

label_names = ['negative', 'neutral', 'positive']

print("Class counts:\n", df['label'].value_counts().sort_index().rename(index=dict(enumerate(label_names))))

# ------------------------------

Class counts:
 label
negative      812
neutral      1499
positive    32315
Name: count, dtype: int64


In [54]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import Dataset

In [55]:
# Cleaning data leaving only what we need

assert 'reviews.text' in df.columns and 'reviews.rating' in df.columns, df.columns
df = df[['reviews.text', 'reviews.rating']].copy()

In [56]:
# Pre-prossesing remove sigbola
def clean_text(s: str) -> str:
    s = re.sub(r'<br\s*/?>', ' ', str(s))
    s = re.sub(r'\s+', ' ', s).strip()
    return s

df['reviews.text'] = df['reviews.text'].fillna('').map(clean_text)
df['reviews.rating'] = pd.to_numeric(df['reviews.rating'], errors='coerce').astype('Int64')
df = df.dropna(subset=['reviews.rating'])
df = df[df['reviews.text'].str.len() > 0]
df = df[df['reviews.rating'].between(1, 5)]
df = df.drop_duplicates(subset=['reviews.text'])

In [57]:
# Map Review Points and labels (0=negative ,1=neutral ,2=positive)
def map_star_to_label(star):
    if star in (1, 2): return 0
    if star == 3:      return 1
    return 2

df['label'] = df['reviews.rating'].map(map_star_to_label).astype(int)
df['text']  = df['reviews.text']


In [58]:
# Training and Split
train_df, temp_df = train_test_split(
    df[['text','label']], test_size=0.2, random_state=42, stratify=df['label']
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.5, random_state=42, stratify=temp_df['label']
)

In [59]:
# Tokenizer + datasets loading model_name = "distilbert-base-uncased"
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch['text'], truncation=True , max_length=256)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True)).map(tokenize, batched=True)
val_ds   = Dataset.from_pandas(val_df.reset_index(drop=True)).map(tokenize, batched=True)
test_ds  = Dataset.from_pandas(test_df.reset_index(drop=True)).map(tokenize, batched=True)

train_ds = train_ds.rename_columns({'label':'labels'})
val_ds   = val_ds.rename_columns({'label':'labels'})
test_ds  = test_ds.rename_columns({'label':'labels'})

data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/27692 [00:00<?, ? examples/s]

Map:   0%|          | 0/3461 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

In [60]:
# Model ----------
id2label = {0: 'negative', 1: 'neutral', 2: 'positive'}
label2id = {v: k for k, v in id2label.items()}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=3, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [61]:
#Evaluations Mtrics

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average='macro'),
        "f1_weighted": f1_score(labels, preds, average='weighted'),
    }


In [ ]:
# ====== DROP-IN: Neutral-boosted oversampling + Focal Loss training ======
import numpy as np, torch
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

label_col = "labels" if "labels" in train_ds.column_names else "label"

# --- Build neutral-boosted oversampled TRAIN DATASET (val/test untouched) ---
y = np.array(train_ds[label_col])
classes = np.array([0,1,2])  # 0=neg, 1=neu, 2=pos
idx_by_class = {c: np.where(y == c)[0] for c in classes}
pos_n, neg_n, neu_n = len(idx_by_class[2]), len(idx_by_class[0]), len(idx_by_class[1])

rng = np.random.default_rng(42)
target_neg = min(pos_n, max(neg_n, int(0.5 * pos_n)))   # ~50% of positives
target_neu = min(pos_n, max(neu_n, int(0.7 * pos_n)))   # ~70% of positives (extra neutral boost)
target_pos = pos_n                                      # keep all positives

oversampled_idx = np.concatenate([
    rng.choice(idx_by_class[0], size=target_neg, replace=True),
    rng.choice(idx_by_class[1], size=target_neu, replace=True),
    rng.choice(idx_by_class[2], size=target_pos, replace=False),
])
rng.shuffle(oversampled_idx)
train_ds_bal = train_ds.select(oversampled_idx)
print("Balanced sizes (neg, neu, pos):", target_neg, target_neu, target_pos, "=> total:", len(train_ds_bal))

# --- Alpha from ORIGINAL (pre-oversample) distribution, then clamp & renorm ---
y_orig = np.array(train_ds[label_col])
weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_orig)
alpha = torch.tensor(weights, dtype=torch.float)
alpha = torch.clamp(alpha, min=0.05)     # avoid near-zero for majority
alpha = alpha / alpha.sum()
print("Alpha:", dict(zip(classes, alpha.tolist())))

# --- Focal Loss (harder focus on neutral/neg) ---
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=None, gamma=2.5, reduction="mean"):  # gamma↑
        super().__init__()
        self.alpha, self.gamma, self.reduction = alpha, gamma, reduction
    def forward(self, logits, targets):
        ce = torch.nn.functional.cross_entropy(logits, targets, reduction="none")
        pt = torch.exp(-ce)
        if self.alpha is not None:
            at = self.alpha.to(logits.device)[targets]
            loss = at * (1 - pt) ** self.gamma * ce
        else:
            loss = (1 - pt) ** self.gamma * ce
        return loss.mean() if self.reduction == "mean" else (loss.sum() if self.reduction == "sum" else loss)

# --- Training args (older-transformers friendly) ---
training_args = TrainingArguments(
    output_dir="./distilbert-sentiment-balanced",
    num_train_epochs=8,                 # a bit longer
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,                 # slightly lower LR helps minorities
    weight_decay=0.01,
    logging_steps=100,
    seed=42,
    report_to="none",
    fp16=torch.cuda.is_available(),
)

data_collator = DataCollatorWithPadding(tokenizer)

def compute_metrics_bal(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
        "f1_weighted": f1_score(labels, preds, average="weighted"),
    }

# Accept extra kwargs to avoid Trainer kw errors
class FocalTrainer(Trainer):
    def __init__(self, focal_loss, *args, **kwargs):
        super().__init__(*args, **kwargs); self.focal_loss = focal_loss
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels  = inputs.get("labels")
        outputs = model(**inputs)
        logits  = outputs.get("logits")
        loss    = self.focal_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

trainer = FocalTrainer(
    focal_loss=FocalLoss(alpha=alpha, gamma=2.5, reduction="mean"),
    model=model,                               # or peft_model if using LoRA
    args=training_args,
    train_dataset=train_ds_bal,                # oversampled TRAIN
    eval_dataset=val_ds,                       # untouched VAL
    data_collator=data_collator,
    tokenizer=tokenizer,                       # use processing_class=tokenizer if your version supports it
    compute_metrics=compute_metrics_bal,
)

trainer.train()

# --- Evaluation (unchanged for your flow) ---
print("\nValidation metrics:", trainer.evaluate(eval_dataset=val_ds))
print("\nTest metrics:", trainer.evaluate(eval_dataset=test_ds))

pred_out   = trainer.predict(test_ds)
test_preds = np.argmax(pred_out.predictions, axis=1)
print("\nClassification report (test):")
print(classification_report(test_df['label'], test_preds, target_names=['negative','neutral','positive'], digits=4))
print("Confusion matrix (rows=true, cols=pred):")
print(confusion_matrix(test_df['label'], test_preds))


Balanced sizes (neg, neu, pos): 12922 18091 25845 => total: 56858
Alpha: {np.int64(0): 0.6387642025947571, np.int64(1): 0.3452203869819641, np.int64(2): 0.016015447676181793}


/tmp/ipython-input-1821279859.py:80: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `FocalTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs); self.focal_loss = focal_loss


Step,Training Loss
100,0.054500
200,0.038800
300,0.032000
400,0.027300
500,0.024300
600,0.021100
700,0.018300
800,0.012700
900,0.013000
1000,0.011700


In [ ]:
# ========= Balanced + Focal Loss (syntax-safe) =========
import numpy as np, torch
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# Figure out labels column
label_col = "labels" if "labels" in train_ds.column_names else "label"

# Original distribution (before oversampling)
y_orig   = np.array(train_ds[label_col])
classes  = np.array([0, 1, 2])  # 0=neg, 1=neu, 2=pos
counts   = np.bincount(y_orig, minlength=3)
print("Original train counts:", dict(zip(classes, counts)))

# Oversample TRAIN ONLY
max_n = counts.max()
rng = np.random.default_rng(42)
idx_by_class = {c: np.where(y_orig == c)[0] for c in classes}
oversampled_idx = np.concatenate([rng.choice(idx_by_class[c], size=max_n, replace=True) for c in classes])
rng.shuffle(oversampled_idx)
train_ds_bal = train_ds.select(oversampled_idx)
print("Balanced train size:", len(train_ds_bal))

# Alpha from original distribution (inverse-freq), clip & renorm to avoid near-zero majority weight
weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_orig)
alpha = torch.tensor(weights, dtype=torch.float)
alpha = torch.clamp(alpha, min=0.05)
alpha = alpha / alpha.sum()
print("Alpha (per-class):", dict(zip(classes, alpha.tolist())))

# Focal Loss
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction="mean"):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, targets):
        ce = torch.nn.functional.cross_entropy(logits, targets, reduction="none")
        pt = torch.exp(-ce)
        if self.alpha is not None:
            at = self.alpha.to(logits.device)[targets]
            loss = at * (1 - pt) ** self.gamma * ce
        else:
            loss = (1 - pt) ** self.gamma * ce
        if self.reduction == "mean":
            return loss.mean()
        elif self.reduction == "sum":
            return loss.sum()
        return loss

# Minimal TrainingArguments (older-transformers friendly)
training_args = TrainingArguments(
    output_dir="./distilbert-sentiment-balanced",
    num_train_epochs=6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    seed=42,
    report_to="none",
    fp16=torch.cuda.is_available()
)

data_collator = DataCollatorWithPadding(tokenizer)

def compute_metrics_bal(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
        "f1_weighted": f1_score(labels, preds, average="weighted"),
    }

# Trainer that accepts extra kwargs (fixes num_items_in_batch issue)
class FocalTrainer(Trainer):
    def __init__(self, focal_loss, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.focal_loss = focal_loss

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels  = inputs.get("labels")
        outputs = model(**inputs)
        logits  = outputs.get("logits")
        loss    = self.focal_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Use tokenizer= for maximum compatibility (processing_class is optional/newer)
trainer = FocalTrainer(
    focal_loss=FocalLoss(alpha=alpha, gamma=2.0, reduction="mean"),
    model=model,                              # or peft_model if using LoRA
    args=training_args,
    train_dataset=train_ds_bal,
    eval_dataset=val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_bal
)

# Train
trainer.train()

# Evaluate
print("\nValidation metrics:", trainer.evaluate(eval_dataset=val_ds))
print("\nTest metrics:", trainer.evaluate(eval_dataset=test_ds))

pred_out   = trainer.predict(test_ds)
test_preds = np.argmax(pred_out.predictions, axis=1)
print("\nClassification report (test):")
print(classification_report(test_df['label'], test_preds, target_names=['negative','neutral','positive'], digits=4))
print("Confusion matrix (rows=true, cols=pred):")
print(confusion_matrix(test_df['label'], test_preds))


In [ ]:


# Evaluate
print("\nValidation metrics:", trainer.evaluate(eval_dataset=val_ds))
print("\nTest metrics:", trainer.evaluate(eval_dataset=test_ds))

pred_out   = trainer.predict(test_ds)
test_preds = np.argmax(pred_out.predictions, axis=1)
print("\nClassification report (test):")
print(classification_report(test_df['label'], test_preds, target_names=['negative','neutral','positive'], digits=4))
print("Confusion matrix (rows=true, cols=pred):")
print(confusion_matrix(test_df['label'], test_preds))
